# Compute single unit metrics for each session

1 - either use all annotated & uploaded ephys sessions as input or provide a list of session_ids

2 - choose whether to overwrite exsiting files

3 - set a savepath and filename for the output - one .pkl file per session

4 - compute metrics

(see description of metrics below)

In [1]:
import pandas as pd
import npc_lims
# from npc_sessions import DynamicRoutingSession
from dynamic_routing_analysis import spike_utils
from dynamic_routing_analysis import data_utils
# import dynamic_routing_analysis as dra
import os
import pickle
import upath

%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [ ]:
#get all uploaded & annotated ephys sessions
# ephys_sessions=tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True,))

In [2]:
#alternatively, provide a list of session ids:
session_id_list=['742903_2024-10-22']
session_list=[]
for ss in session_id_list:
    session_list.append(npc_lims.get_session_info(ss))
ephys_sessions=tuple(session_list)

In [13]:
except_dict={}
overwrite_files=False
# save_path = dra.SINGLE_UNIT_METRICS_PATH
save_path= upath.UPath(r'D:\test_single_unit_metrics')

for session_info in ephys_sessions[:]:
    
    # try:
        trials=[]
        units=[]
        ##skip if file already exists
        
        if overwrite_files==False and (save_path / f"{session_info.id}_stim_context_modulation.pkl").exists():
            print('session',session_info.id,'already processed; skipping')
            continue
        lick_save_path=os.path.join(save_path,'lick_modulation')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # try:
        # session=DynamicRoutingSession(session_info.id)
        session_id=session_info.id
        trials=data_utils.load_trials_or_units(session_id, 'trials', version='0.0.272')
        units=data_utils.load_trials_or_units(session_id, 'units', version='0.0.272')

        # except Exception as e:
        #     print('failed to load trials or units, skipping session')
        #     except_dict[session_info.id]=e
        #     continue

        lick_mod=spike_utils.compute_lick_modulation(trials, units, session_info, save_path=None, test=True)

        stim_mod=spike_utils.compute_stim_context_modulation(trials, units, session_info, save_path=None, test=True)

        merged_df=stim_mod.merge(lick_mod, on=['unit_id','project'], how='left')

        merged_df.to_parquet(save_path / f"{session_info.id}_stim_context_lick_modulation_refactor.parquet")

        # print(session_info.id,'done')

    # except Exception as e:
    #     print(session_info.id,'failed')
    #     except_dict[session_info.id]=e


742903_2024-10-22_0 cached trials loaded
742903_2024-10-22_0 cached units loaded


C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:642: RuntimeWarning: divide by zero encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:661: RuntimeWarning: divide by zero encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:676: RuntimeWarning: invalid value encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:689: RuntimeWarning: invalid value encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:696: RuntimeWarning: invalid value encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src\dynamic_routing_analysis\spike_utils.py:706: RuntimeWarning: invalid value encountered in divide
  
C:\Users\ethan.mcbride\Code\dynamic_routing_analysis\src

In [ ]:
except_dict

In [17]:
# stim_mod.merge(lick_mod, on=['unit_id','project'], how='left')
# stim_mod['session_id']
test_units=pd.read_parquet('s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.272/consolidated/units_with_activity_metrics_exclude_instruction_trials.parquet')

In [18]:
test_units

,index,activity_drift,amplitude,amplitude_cutoff,amplitude_cv_median,amplitude_cv_range,amplitude_median,ccf_ap,ccf_dv,ccf_ml,...,sound1_stimulus_late_modulation_roc_auc,sound1_context_modulation_roc_auc,sound1_evoked_context_modulation_roc_auc,sound1_stimulus_late_context_modulation_roc_auc,sound1_evoked_stimulus_late_context_modulation_roc_auc,lick_modulation_zscore,lick_modulation_index,lick_modulation_p_value,lick_modulation_sign,lick_modulation_roc_auc
0,0,0.287755,201.829674,0.000009,NaN,NaN,119.340000,6100.0,3625.0,4550.0,...,0.908145,0.585818,0.468684,0.543812,0.468759,2.444300,0.523258,4.050993e-43,1.0,0.854670
1,1,0.243794,248.077496,0.003430,NaN,NaN,194.219990,6100.0,3625.0,4550.0,...,0.543020,0.488106,0.488257,0.537940,0.536585,0.477915,0.505909,3.035710e-06,1.0,0.552235
2,10,0.729647,140.708870,0.000175,NaN,NaN,86.579994,6100.0,3600.0,4550.0,...,0.527984,0.455059,0.468684,0.420506,0.441057,0.056898,0.055173,1.879035e-02,1.0,0.540336
3,97,0.097291,80.758121,0.000014,0.225520,0.111005,53.820000,5700.0,1025.0,4650.0,...,0.237965,0.495559,0.520175,0.476513,0.485245,-0.678844,-0.346051,1.104374e-13,-1.0,0.310316
4,98,0.326657,71.791208,0.000240,0.239983,0.121313,49.140000,5700.0,1025.0,4650.0,...,0.770955,0.452198,0.442111,0.479825,0.466576,0.692294,0.347870,2.799237e-14,1.0,0.684454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446609,3775,0.086122,204.300610,0.000053,0.163228,0.162902,152.099990,3250.0,3800.0,5350.0,...,0.627000,0.569550,0.530392,0.616452,0.579340,0.097836,0.082014,8.956569e-01,1.0,0.507253
446610,3776,0.138864,143.006752,0.000011,0.214832,0.167319,91.259995,3250.0,3800.0,5350.0,...,0.565491,0.572034,0.474722,0.585476,0.527323,-0.259337,-0.110864,1.597807e-01,-1.0,0.404014
446611,3777,0.077887,119.653568,0.000084,0.297627,0.158574,79.560000,3250.0,3800.0,5350.0,...,0.547301,0.502192,0.467417,0.614699,0.556400,0.188975,0.101563,2.944931e-01,1.0,0.569149
446612,3778,0.068722,482.929321,0.000007,0.176435,0.149985,332.280000,3250.0,3750.0,5350.0,...,0.608262,0.645529,0.667738,0.513442,0.588252,0.576165,0.117642,1.072439e-01,1.0,0.609768


## single unit metrics descriptions


### baseline context modulation

comparisons between all visual context vs. auditory context trials

time window = [-0.1, 0] relative to stimulus onset

**'baseline_context_modulation_index'** - (vis-aud)/(vis+aud)

**'baseline_context_modulation_p_value'** - scipy.stats.mannwhitneyu(vis,aud)

**'baseline_context_modulation_zscore'** - (vis-aud)/std(all trials)

**'baseline_context_modulation_sign'** - np.sign(np.mean(vis-aud))

**'baseline_context_roc_auc'** - sklearn.metrics.roc_auc_score(context_label,baseline_frs)

**'linear_shift_baseline_context_true_value'** - mean (vis-aud) baseline fr difference, properly aligned

**'linear_shift_baseline_context_null_median'** - median (vis-aud) across all shifts

**'linear_shift_baseline_context_null_mean'** - mean (vis-aud) across all shifts

'**linear_shift_baseline_context_null_std**' - std of (vis-aud) across all shifts

'**linear_shift_baseline_context_p_value_higher**' - the fraction of shift values that the true value is higher than

'**linear_shift_baseline_context_p_value_lower**' - the fraction of shift values that the true value is lower than



### stimulus modulation

comparisons between baseline and stimulus firing rates per trial

baseline time window = [-0.1,0] sec relative to stimulus onset

stimulus time window = [0,0.1]

late stimulus time window = [0.1,0.2]

replace "stim" with specific stimulus name: vis1, vis2, sound1, sound2, or catch

'**stim_stimulus_modulation_index**' - (stim-baseline)/(stim+baseline)

'**stim_stimulus_modulation_zscore**' - (stim-baseline)/std(baseline)

'**stim_stimulus_modulation_p_value**' - scipy.stats.wilcoxon(stim,baseline)

'**stim_stimulus_modulation_sign**' - np.sign(np.mean(stim-baseline))

'**stim_stimulus_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(stim_vs_baseline_label,frs)

'**stim_stimulus_late_modulation_index**' - same as above but for later time window

'**stim_stimulus_late_modulation_zscore**' 

'**stim_stimulus_late_modulation_p_value**'

'**stim_stimulus_late_modulation_sign**'

'**stim_stimulus_late_modulation_roc_auc**'

'**stim_stim_latency**' - latency of the maximum response in the [0,0.3] time window - in units of binsize (default=0.025sec)



### context modulation of stimulus response

comparison between stimulus responses in same vs. other contexts

(i.e. for vis1, same context = vis, other context = aud)

time window = [0,0.1] relative to stimulus onset

'**stim_context_modulation_index**' - (stim_same_context-stim_other_context)/(stim_same_context+stim_other_context)

'**stim_context_modulation_zscore**' - (stim_same_context-stim_other_context)/(stim_same_context+stim_other_context)

'**stim_context_modulation_sign**' - np.sign(np.mean(stim_same_context-stim_other_context))

'**stim_context_modulation_p_value**' - scipy.stats.mannwhitneyu(stim_same_context,stim_other_context)

'**stim_context_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(context_label,stim_frs)

'**stim_evoked_context_modulation_index**' - same as above but for evoked fr - subtract the [-0.1,0] baseline per trial

'**stim_evoked_context_modulation_zscore**'

'**stim_evoked_context_modulation_sign**'

'**stim_evoked_context_modulation_p_value**'



### lick modulation

comparisons between correct reject (no_lick) and false alarm (lick) trials

time window: [0.2,0.5]

baseline: [-0.5,-0.2]

'**lick_modulation_index**' - (lick-no_lick)/(lick+no_lick)

'**lick_modulation_zscore**' - (lick-no_lick)/std(baseline)

'**lick_modulation_p_value**' - st.mannwhitneyu(lick,no_lick)

'**lick_modulation_sign**' - np.sign(np.mean(lick-no_lick))

'**lick_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(lick, no_lick)



### stimulus selectivity

comparisons between firing rate responses to different stimuli

time window: [0,0.1]

'**vis_discrim_roc_auc**' - vis1 vs. vis2

'**aud_discrim_roc_auc**' - sound1 vs. sound2

'**target_discrim_roc_auc**' - vis1 vs. sound1

'**nontarget_discrim_roc_auc**' - vis2 vs. sound2

'**vis_vs_aud**' - all vis vs. all sound


### trial response type selectivity

comparisons between response types, in different time windows

all stimuli included - maybe should be

time windows [0,0.1] [0.1,0.2] [0.2,0.3]

'cr_vs_fa_early_roc_auc'

'hit_vs_cr_early_roc_auc'

'hit_vs_fa_early_roc_auc'

'cr_vs_fa_mid_roc_auc'

'hit_vs_cr_mid_roc_auc'

'hit_vs_fa_mid_roc_auc'

'cr_vs_fa_late_roc_auc'

'hit_vs_cr_late_roc_auc'

'hit_vs_fa_late_roc_auc'
